In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]

tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
import pandas as pd
df = pd.read_csv('fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
# Get independent features
X =df.drop('label', axis=1)

In [6]:
# Get dependent features
y = df['label']

In [7]:
X.shape, y.shape

((18285, 4), (18285,))

In [8]:
import tensorflow as tf

In [9]:
tf.__version__

'2.5.0'

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
## Vocabulary size
voc_size= 5000

## OneHot Representation

In [12]:
messages = X.copy()

In [13]:
messages.reset_index(inplace=True)

In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


False

In [16]:
## Data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

## One Hot Representation

In [17]:
onehot_repr = [one_hot(words , voc_size) for words in corpus]

In [18]:
onehot_repr

[[3141, 75, 2686, 2496, 2974, 480, 2806, 3933, 1607, 1421],
 [2012, 3975, 1835, 792, 1224, 918, 699],
 [1224, 2088, 2590, 3686],
 [320, 3757, 4749, 2041, 1619, 2030],
 [2874, 1224, 2211, 1595, 3366, 3354, 1224, 1609, 3381, 1084],
 [3763,
  4715,
  834,
  2613,
  3284,
  2134,
  873,
  64,
  2787,
  3376,
  3761,
  2062,
  133,
  4935,
  699],
 [1262, 3473, 4464, 3380, 1130, 2563, 3680, 904, 2887, 4508, 1183],
 [93, 1900, 3832, 2333, 1028, 788, 2134, 2534, 2887, 4508, 1183],
 [3175, 613, 3628, 2498, 3166, 3766, 4953, 4061, 2134, 2413],
 [210, 422, 1152, 1120, 3056, 4197, 1696, 4269],
 [1210, 435, 3580, 2411, 2293, 3673, 4909, 1051, 1947, 1018, 4116],
 [2041, 4928, 2974, 3766, 2134, 1028],
 [588, 627, 2592, 4685, 3190, 3816, 3208, 1617, 4805],
 [3158, 1161, 2096, 759, 3657, 2470, 1224, 2887, 4508, 1183],
 [955, 3539, 3907, 3038, 3614, 2887, 4508, 1183],
 [2873, 3693, 1022, 4147, 1880, 4739, 1151, 2276, 761, 1392],
 [565, 4712, 3975],
 [4284, 3905, 4263, 1049, 2134, 519, 1676, 699],
 [319

In [19]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3933 1607 1421]
 [   0    0    0 ... 1224  918  699]
 [   0    0    0 ... 2088 2590 3686]
 ...
 [   0    0    0 ... 2887 4508 1183]
 [   0    0    0 ... 1165 1528 4630]
 [   0    0    0 ... 2273  221 1253]]


In [20]:
len(embedded_docs)

18285

## Creating model

In [21]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# 100 neurons
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [23]:
len(embedded_docs),  y.shape

(18285, (18285,))

In [24]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [25]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final,test_size=0.33,random_state=42)

## Model Training

In [41]:
# TF_FORCE_GPU_ALLOW_GROWTH=1
# tf.config.experimental.set_memory_growth(gpu, True)


In [42]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [43]:
model.fit(X_train,y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)

Epoch 1/10


CancelledError:  [_Derived_]RecvAsync is cancelled.
	 [[{{node Adam/Adam/update/AssignSubVariableOp/_39}}]]
	 [[gradient_tape/sequential/embedding/embedding_lookup/Reshape/_36]] [Op:__inference_train_function_3212]

Function call stack:
train_function


## Performance Metrics and Accuracy

In [44]:
y_pred = model.predict_classes(X_test)

/home/gourav/.virtualenvs/spark/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_predict_function_6684]

Function call stack:
predict_function -> predict_function -> predict_function


In [45]:
from sklearn.metrics import confusion_matrix

In [46]:
confusion_matrix(y_test, y_pred=y_pred)

NameError: name 'y_pred' is not defined

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_te)